<a href="https://colab.research.google.com/github/yepez26/SBBD2024_FUNK/blob/main/identificacao_topicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install maritalk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import maritalk
import random
import IPython
# from IPython.core.debugger import set_trace


In [5]:
# Clonar o repositório
!git clone https://github.com/yepez26/SBBD2024_FUNK
# Mudar para o diretório do repositório
%cd SBBD2024_FUNK


# Load the Excel file
file_path = 'funk_lyrics.xlsx'
data = pd.read_excel(file_path)

# Display the first few rows of the file
data.head()

Cloning into 'SBBD2024_FUNK'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 8 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), 125.79 KiB | 780.00 KiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/SBBD2024_FUNK


,Unnamed: 0,ESTROFE,musica,artista,text_original,text,ntokens
0,0,Nopa Flagra aquela mina Mó piriguete véi É S...,Piriguete,MC Papo,Nopa Flagra aquela mina Mó piriguete véi É S...,nopa flagra aquela mina mo piriguete vei e spi...,15
1,1,"Quando ela me vê, ela mexe Piri, pipiri, pipir...",Piriguete,MC Papo,"Quando ela me vê, ela mexe Piri, pipiri, pipir...",quando ela me ve ela mexe piri pipiri pipiri p...,20
2,2,"Quando ela me vê, ela mexe Piri, pipiri, pipir...",Piriguete,MC Papo,"Quando ela me vê, ela mexe Piri, pipiri, pipir...",quando ela me ve ela mexe piri pipiri pipiri p...,20
3,3,Mini saia rodada Blusa rosinha Decote enfeitad...,Piriguete,MC Papo,Mini saia rodada Blusa rosinha Decote enfeitad...,mini saia rodada blusa rosinha decote enfeitad...,12
4,4,Ela não paga Ganha cortesia Foge se a sua Cart...,Piriguete,MC Papo,Ela não paga Ganha cortesia Foge se a sua Cart...,ela nao paga ganha cortesia foge se a sua cart...,12


In [ ]:
#parameters

num_iter = 10 # numero de vezes que será repetida a interação
num_groups = 20 # numero de grupos diferente que será separada cada interação
num_topicos = 5 # numero de tópicos que será perguntado o modelo
aux_dic = {number: ['topics:'] for number in range(0,num_topicos)} # dicionario auxiliar para montagem de base final
tamanho = len(lyrics_test)

In [ ]:
# definindo os do prompt

model = maritalk.MariTalk(key="")
# dados os seguintes trechos de música, sugira {n} tópicos que descrevam os assuntos abordados
template1 = "dados os seguintes trechos de música, sugira {} tópicos que descrevam os assuntos abordados: ".format(num_topicos)

In [ ]:
# %xmode Verbose
# %pdb on
path_save = ''
lista_dfs = [] # lista onde vão ser salvo os dfs de cada iteração

for iter in range(0,num_iter):

    random.seed(iter*10)
    df_1_volta = pd.DataFrame() # df com o resultado final de topicos ao longo de 1 iteração
    # definido os trechos de comparação aletóriamente usando a biblioteca random
    list_sample = []
    for i in range(0,tamanho):
      list_sample.append(random.randrange(0,num_groups))

    lyrics_test['sample'] = list_sample

    # fazendo a iteração dentro dos grupos criados
    # df_aux1 = pd.DataFrame()
    for grupo in range(0,num_groups):

        # filtrar musicas do grupo
        lista_trechos = list(lyrics_test[lyrics_test['sample']==grupo]['trecho'].reset_index(drop=True))

        # string com trechos músicas
        string_trechos = ['trecho {}: '.format(i) + lista_trechos[i] for i in range(0,len(lista_trechos))]
        string_trechos = '; '.join(string_trechos) + ' '

        #template com frases do grupo
        template2 = template1 + string_trechos

        # chamda modelo
        answer = model.generate(
            template2,
          do_sample=True,
            # max_tokens=200,
            temperature=0.5,
            top_p=0.95)

        # splitar respostas em lista
        list_answer = answer.split('\n')


        list_answer = list_answer[0:num_topicos]

        while len(list_answer) <  num_topicos:
            list_answer.append('sem_topico')


        # df auxiliar final
        df_aux = pd.DataFrame()
        df_aux['trechos'] = lista_trechos

        for col in aux_dic.keys():
          df_aux[col] = str(aux_dic[col])

        for i in range(0,num_topicos):
          # concat horizontal two list
          df_aux[i] = df_aux[i]+str(list_answer[i])

        df_1_volta = df_1_volta.append(df_aux)
        # set_trace()

    df_1_volta.to_excel(path_save+'/base_topics_{}.xlsx'.format(iter))
    lista_dfs.append([df_1_volta])
    # set_trace()



